In [1]:
# Cell 1: Install Dependencies
!pip install gradio opencv-python-headless matplotlib ultralytics supervision -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 22.6 MB/s eta 0:00:00


In [2]:
# Cell 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Cell 3: Import Libraries
import cv2
import numpy as np
import torch
import torch.nn as nn
from torchvision import models
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import cm
import gradio as gr
from PIL import Image
import time
from collections import defaultdict
from ultralytics import YOLO
import supervision as sv
from datetime import datetime
import os

# Check device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✓ Using device: {DEVICE}")

# Model training parameters (MUST match your training config)
TARGET_SIZE = (512, 512)  # (Width, Height)
GT_DOWNSAMPLE = 8

# ImageNet normalization constants
IMAGENET_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
IMAGENET_STD = np.array([0.229, 0.224, 0.225], dtype=np.float32)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✓ Using device: cuda


In [5]:
# Cell 4: CSRNet Model Architecture
def create_csrnet():
    """Create CSRNet model architecture"""
    print("Building CSRNet architecture...")

    vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
    features = list(vgg.features.children())

    frontend = nn.Sequential(*features[0:23])

    backend = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size=3, padding=1, dilation=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(512, 512, kernel_size=3, padding=2, dilation=2),
        nn.ReLU(inplace=True),
        nn.Conv2d(512, 512, kernel_size=3, padding=4, dilation=4),
        nn.ReLU(inplace=True),
        nn.Conv2d(512, 256, kernel_size=3, padding=1, dilation=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(256, 128, kernel_size=3, padding=1, dilation=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(128, 1, kernel_size=1, padding=0),
    )

    model = nn.Sequential(frontend, backend)
    print("✓ CSRNet architecture created")

    return model


def load_trained_model(model_path, device=DEVICE):
    """Load trained CSRNet model"""
    print(f"Loading CSRNet from: {model_path}")

    model = create_csrnet()
    checkpoint = torch.load(model_path, map_location=device)

    if 'model_state' in checkpoint:
        model.load_state_dict(checkpoint['model_state'])
        print(f"✓ Loaded checkpoint from epoch {checkpoint.get('epoch', 'unknown')}")
    else:
        model.load_state_dict(checkpoint)
        print("✓ Loaded model weights")

    model.to(device)
    model.eval()

    print(f"✓ CSRNet ready on {device}")
    return model

In [6]:
# Cell 5: Person Tracker Class
class PersonTracker:
    """
    Multi-object tracker to prevent re-counting same people
    Uses ByteTrack algorithm via supervision library
    """

    def __init__(self):
        # Initialize ByteTrack tracker
        self.tracker = sv.ByteTrack(
            track_activation_threshold=0.25,
            lost_track_buffer=30,
            minimum_matching_threshold=0.8,
            frame_rate=30
        )

        # Track statistics
        self.unique_ids = set()
        self.track_history = defaultdict(list)

    def update(self, detections):
        """
        Update tracker with new detections
        Args:
            detections: supervision Detections object
        Returns:
            tracked_detections: Detections with tracker_id
        """
        tracked_detections = self.tracker.update_with_detections(detections)

        # Update unique IDs
        if tracked_detections.tracker_id is not None:
            for track_id in tracked_detections.tracker_id:
                self.unique_ids.add(int(track_id))

        return tracked_detections

    def get_total_unique_count(self):
        """Get total unique people seen"""
        return len(self.unique_ids)

    def reset(self):
        """Reset tracker"""
        self.tracker = sv.ByteTrack(
            track_activation_threshold=0.25,
            lost_track_buffer=30,
            minimum_matching_threshold=0.8,
            frame_rate=30
        )
        self.unique_ids = set()
        self.track_history.clear()

In [7]:
# Cell 6: Enhanced Crowd Counter with Detection
class EnhancedCrowdCounter:
    """
    Enhanced counter with both density estimation (CSRNet)
    and object detection (YOLOv8) + tracking
    """

    def __init__(self, csrnet_model, device=DEVICE):
        self.csrnet = csrnet_model
        self.device = device
        self.mean = IMAGENET_MEAN
        self.std = IMAGENET_STD
        self.target_size = TARGET_SIZE

        # Load YOLOv8 for person detection
        print("Loading YOLOv8 person detector...")
        self.yolo = YOLO('yolov8n.pt')  # Nano model for speed
        print("✓ YOLOv8 loaded")

        # Initialize tracker
        self.tracker = PersonTracker()

        # Annotators for visualization
        self.box_annotator = sv.BoxAnnotator(thickness=2)
        self.trace_annotator = sv.TraceAnnotator(thickness=2, trace_length=50)

    def preprocess_frame(self, frame):
        """Preprocess frame for CSRNet - resize to training size"""
        # Resize to TARGET_SIZE used during training
        frame_resized = cv2.resize(frame, self.target_size, interpolation=cv2.INTER_LINEAR)

        img_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
        img_normalized = img_rgb.astype(np.float32) / 255.0
        img_normalized = (img_normalized - self.mean) / self.std
        img_tensor = torch.from_numpy(img_normalized).permute(2, 0, 1).unsqueeze(0)
        img_tensor = img_tensor.to(self.device, dtype=torch.float32)
        return img_tensor

    def predict_density(self, frame):
        """Get density map from CSRNet"""
        with torch.no_grad():
            img_tensor = self.preprocess_frame(frame)
            density_map = self.csrnet(img_tensor)
            density_np = density_map.squeeze().cpu().numpy()
            count = float(density_np.sum())
        return density_np, count

    def detect_people(self, frame):
        """
        Detect people using YOLOv8
        Returns: supervision Detections object
        """
        results = self.yolo(frame, classes=[0], verbose=False)[0]  # class 0 = person
        detections = sv.Detections.from_ultralytics(results)
        return detections

    def predict_with_tracking(self, frame):
        """
        Complete prediction pipeline:
        1. Detect people with YOLOv8
        2. Track people to avoid re-counting
        3. Get density map from CSRNet

        Returns:
            annotated_frame: Frame with bounding boxes and IDs
            density_map: CSRNet density map
            detection_count: Current frame person count
            unique_count: Total unique people tracked
            density_count: CSRNet estimate
        """
        # Detect people
        detections = self.detect_people(frame)

        # Track people
        tracked_detections = self.tracker.update(detections)

        # Get counts
        detection_count = len(tracked_detections)
        unique_count = self.tracker.get_total_unique_count()

        # Get density map
        density_map, density_count = self.predict_density(frame)

        # Annotate frame
        annotated_frame = frame.copy()

        # Draw bounding boxes and labels
        if len(tracked_detections) > 0:
            # Draw boxes first
            annotated_frame = self.box_annotator.annotate(
                scene=annotated_frame,
                detections=tracked_detections
            )

            # Draw tracking traces
            annotated_frame = self.trace_annotator.annotate(
                scene=annotated_frame,
                detections=tracked_detections
            )

            # Draw labels manually using OpenCV
            if tracked_detections.tracker_id is not None:
                for i, (bbox, track_id) in enumerate(zip(tracked_detections.xyxy, tracked_detections.tracker_id)):
                    x1, y1, x2, y2 = map(int, bbox)
                    label = f"ID:{track_id}"

                    # Draw label background
                    label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
                    cv2.rectangle(annotated_frame,
                                (x1, y1 - label_size[1] - 10),
                                (x1 + label_size[0], y1),
                                (0, 255, 0), -1)

                    # Draw label text
                    cv2.putText(annotated_frame, label,
                              (x1, y1 - 5),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

        return annotated_frame, density_map, detection_count, unique_count, density_count

    def create_heatmap_overlay(self, density_map, original_frame, alpha=0.5):
        """Create density heatmap overlay"""
        h, w = original_frame.shape[:2]
        density_resized = cv2.resize(density_map, (w, h), interpolation=cv2.INTER_CUBIC)

        density_max = density_resized.max()
        if density_max > 0:
            density_normalized = density_resized / density_max
        else:
            density_normalized = density_resized

        heatmap = cm.jet(density_normalized)[:, :, :3]
        heatmap = (heatmap * 255).astype(np.uint8)

        overlay = cv2.addWeighted(original_frame, 1-alpha, heatmap, alpha, 0)

        return overlay, heatmap

    def reset_tracker(self):
        """Reset tracker for new video"""
        self.tracker.reset()

In [8]:
# Cell 7: Video Processing with Save Functionality
def process_video_with_save(video_path, counter, alert_threshold=50, frame_skip=1,
                            output_path=None, save_video=True):
    """
    Process video file with detection + tracking + density
    Save annotated output video
    """
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise ValueError(f"Cannot open video: {video_path}")

    # Reset tracker for new video
    counter.reset_tracker()

    # Get video properties
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Setup output video writer
    if save_video:
        if output_path is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_path = f"/content/output_annotated_{timestamp}.mp4"

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        print(f"📹 Saving output to: {output_path}")

    frame_idx = 0
    processed_idx = 0

    # Statistics
    detection_counts = []
    density_counts = []
    alert_frames = []

    print(f"Processing video: {total_frames} frames @ {fps:.1f} FPS")
    print(f"Frame skip: {frame_skip}")

    start_time = time.time()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1

        if frame_idx % frame_skip != 0:
            continue

        # Process frame with detection + tracking + density
        annotated, density_map, det_count, unique_count, dens_count = counter.predict_with_tracking(frame)

        # Create heatmap overlay
        overlay, heatmap = counter.create_heatmap_overlay(density_map, annotated, alpha=0.3)

        # Add info text overlay
        info_y = 30
        cv2.rectangle(overlay, (10, 10), (400, 120), (0, 0, 0), -1)
        cv2.putText(overlay, f"Frame: {frame_idx}/{total_frames}", (20, info_y),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(overlay, f"Detected: {det_count} | Unique: {unique_count}", (20, info_y + 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(overlay, f"Density Est: {int(dens_count)}", (20, info_y + 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Alert check
        is_alert = det_count > alert_threshold
        if is_alert:
            alert_frames.append(frame_idx)
            # Add alert banner
            cv2.rectangle(overlay, (0, height - 50), (width, height), (0, 0, 255), -1)
            cv2.putText(overlay, f"ALERT! Count: {det_count} > Threshold: {alert_threshold}",
                       (20, height - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        # Write frame to output video
        if save_video:
            out.write(overlay)

        # Update statistics
        detection_counts.append(det_count)
        density_counts.append(dens_count)

        processed_idx += 1

        # Progress update every 30 frames
        if processed_idx % 30 == 0:
            elapsed = time.time() - start_time
            fps_processing = processed_idx / elapsed
            eta = (total_frames // frame_skip - processed_idx) / fps_processing
            print(f"Progress: {processed_idx}/{total_frames // frame_skip} frames | "
                  f"FPS: {fps_processing:.1f} | ETA: {eta:.1f}s")

    cap.release()
    if save_video:
        out.release()

    # Final statistics
    avg_det = np.mean(detection_counts) if detection_counts else 0
    max_det = max(detection_counts) if detection_counts else 0
    avg_dens = np.mean(density_counts) if density_counts else 0

    print(f"\n{'='*60}")
    print(f"✓ Processing complete: {processed_idx} frames")
    print(f"  Total unique people: {unique_count}")
    print(f"  Avg detection: {avg_det:.1f}")
    print(f"  Max detection: {max_det}")
    print(f"  Avg density: {avg_dens:.1f}")
    print(f"  Alert frames: {len(alert_frames)}")
    if save_video:
        print(f"  Output saved: {output_path}")
    print(f"{'='*60}\n")

    return {
        'output_path': output_path if save_video else None,
        'unique_count': unique_count,
        'avg_detection': avg_det,
        'max_detection': max_det,
        'avg_density': avg_dens,
        'alert_frames': alert_frames,
        'processed_frames': processed_idx
    }


In [9]:
# Cell 8: Gradio Interface for Video Processing
def create_video_interface(model_path):
    """Create Gradio interface for video processing"""

    print("="*60)
    print("Loading models...")
    print("="*60)

    # Load CSRNet
    csrnet = load_trained_model(model_path)

    # Create enhanced counter
    counter = EnhancedCrowdCounter(csrnet)

    print("✓ All models ready for inference\n")

    # Video processing function for Gradio
    def process_video_gradio(video_file, threshold, skip_frames, save_output):
        if video_file is None:
            return None, "❌ No video uploaded", ""

        try:
            # Generate output path
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_path = f"/content/output_{timestamp}.mp4" if save_output else None

            # Process video
            stats = process_video_with_save(
                video_file,
                counter,
                alert_threshold=int(threshold),
                frame_skip=int(skip_frames),
                output_path=output_path,
                save_video=save_output
            )

            # Create stats message
            stats_msg = f"""
✓ Processing Complete!

📊 Statistics:
- Processed Frames: {stats['processed_frames']}
- Total Unique People: {stats['unique_count']}
- Average Detection: {stats['avg_detection']:.1f}
- Max Detection: {stats['max_detection']}
- Average Density: {stats['avg_density']:.1f}
- Alert Frames: {len(stats['alert_frames'])}
"""

            if save_output and stats['output_path']:
                stats_msg += f"\n💾 Output Video: {stats['output_path']}"
                # Also save to Google Drive for easy access
                drive_output = f"/content/drive/MyDrive/Testing/outputs/Processed video/{os.path.basename(stats['output_path'])}"
                os.makedirs(os.path.dirname(drive_output), exist_ok=True)
                import shutil
                shutil.copy(stats['output_path'], drive_output)
                stats_msg += f"\n💾 Also saved to Drive: {drive_output}"
                print(f"✓ Video saved to: {stats['output_path']}")
                print(f"✓ Also copied to Drive: {drive_output}")
                return stats['output_path'], "✓ Video processed successfully!", stats_msg
            else:
                return None, "✓ Video processed successfully!", stats_msg

        except Exception as e:
            import traceback
            traceback.print_exc()
            return None, f"❌ Error: {str(e)}", ""

    # Build Gradio UI
    with gr.Blocks(title="CSRNet Video Processor") as demo:

        gr.Markdown("""
        # 🎥 CSRNet Crowd Counter - Video Processor

        **Process MP4/MOV videos with:**
        - 🎯 YOLOv8 Person Detection & Tracking
        - 🗺️ CSRNet Density Heatmaps
        - 🚨 Threshold-based Alerts
        - 💾 Save Annotated Output Video
        """)

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 📤 Input")

                video_input = gr.Video(
                    label="Upload Video (MP4/MOV)",
                    sources=["upload"]
                )

                gr.Markdown("### ⚙️ Settings")

                threshold_slider = gr.Slider(
                    minimum=5,
                    maximum=100,
                    value=30,
                    step=5,
                    label="Alert Threshold",
                    info="Trigger alert when detected count exceeds this"
                )

                skip_frames_slider = gr.Slider(
                    minimum=1,
                    maximum=10,
                    value=2,
                    step=1,
                    label="Frame Skip",
                    info="Process every Nth frame (higher = faster)"
                )

                save_checkbox = gr.Checkbox(
                    label="Save Output Video",
                    value=True,
                    info="Save annotated video to /content/"
                )

                process_btn = gr.Button(
                    "🚀 Process Video",
                    variant="primary",
                    size="lg"
                )

            with gr.Column(scale=2):
                gr.Markdown("### 📊 Results")

                output_video = gr.Video(
                    label="Processed Video Output"
                )

                status_text = gr.Textbox(
                    label="🔔 Status",
                    interactive=False
                )

                stats_text = gr.Textbox(
                    label="📈 Statistics",
                    interactive=False,
                    lines=10
                )

        process_btn.click(
            fn=process_video_gradio,
            inputs=[video_input, threshold_slider, skip_frames_slider, save_checkbox],
            outputs=[output_video, status_text, stats_text]
        )

        gr.Markdown("""
        ---
        ## 📖 How It Works

        ### 🎯 Detection & Tracking
        - **YOLOv8** detects people and draws bounding boxes
        - **ByteTrack** assigns unique IDs to prevent re-counting
        - Green boxes and tracking lines show movement paths

        ### 🗺️ Density Heatmap
        - **CSRNet** provides density estimation overlay
        - Blue = low density, Red = high density
        - Trained on 512x512 images with GT_DOWNSAMPLE=8

        ### 🚨 Alerts
        - Red banner appears when count > threshold
        - Alert frames are logged for review

        ### 💡 Tips
        - Frame skip 2-3 for good speed/accuracy balance
        - Lower threshold for crowded areas (10-20)
        - Higher threshold for sparse areas (30-50)
        - Output videos saved to /content/ folder
        """)

    return demo

In [10]:
# Cell 9: Launch Interface
if __name__ == "__main__":
    # ⚙️ CONFIGURE YOUR MODEL PATH HERE
    MODEL_PATH = "/content/drive/MyDrive/Testing/outputs/best_crowd_counter_unified.pth"

    if not Path(MODEL_PATH).exists():
        print("="*60)
        print("❌ MODEL NOT FOUND")
        print("="*60)
        print(f"Expected path: {MODEL_PATH}")
        print("\nPlease update MODEL_PATH to point to your trained model.")
        print("="*60)
    else:
        print("="*60)
        print("🚀 LAUNCHING VIDEO PROCESSOR")
        print("="*60)

        demo = create_video_interface(MODEL_PATH)

        demo.launch(
            share=True,
            debug=True,
            server_name="0.0.0.0",
            server_port=7860
        )

🚀 LAUNCHING VIDEO PROCESSOR
Loading models...
Loading CSRNet from: /content/drive/MyDrive/Testing/outputs/best_crowd_counter_unified.pth
Building CSRNet architecture...
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:03<00:00, 143MB/s]


✓ CSRNet architecture created
✓ Loaded checkpoint from epoch 191
✓ CSRNet ready on cuda
Loading YOLOv8 person detector...
✓ YOLOv8 loaded
✓ All models ready for inference

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://73a823411ceb1940bb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


📹 Saving output to: /content/output_20251213_073820.mp4
Processing video: 170 frames @ 30.0 FPS
Frame skip: 1
Progress: 30/170 frames | FPS: 4.2 | ETA: 33.7s
Progress: 60/170 frames | FPS: 4.4 | ETA: 25.0s
Progress: 90/170 frames | FPS: 4.7 | ETA: 17.0s
Progress: 120/170 frames | FPS: 4.7 | ETA: 10.6s
Progress: 150/170 frames | FPS: 4.8 | ETA: 4.2s

✓ Processing complete: 170 frames
  Total unique people: 79
  Avg detection: 10.0
  Max detection: 15
  Avg density: 428.0
  Alert frames: 0
  Output saved: /content/output_20251213_073820.mp4

✓ Video saved to: /content/output_20251213_073820.mp4
✓ Also copied to Drive: /content/drive/MyDrive/Testing/outputs/Processed video/output_20251213_073820.mp4


📹 Saving output to: /content/output_20251213_074247.mp4
Processing video: 403 frames @ 30.0 FPS
Frame skip: 1
Progress: 30/403 frames | FPS: 4.9 | ETA: 76.6s
Progress: 60/403 frames | FPS: 5.4 | ETA: 63.8s
Progress: 90/403 frames | FPS: 5.6 | ETA: 55.4s
Progress: 120/403 frames | FPS: 5.5 | ETA: 51.5s
Progress: 150/403 frames | FPS: 5.6 | ETA: 45.1s
Progress: 180/403 frames | FPS: 5.6 | ETA: 39.9s
Progress: 210/403 frames | FPS: 5.6 | ETA: 34.5s
Progress: 240/403 frames | FPS: 5.7 | ETA: 28.8s
Progress: 270/403 frames | FPS: 5.6 | ETA: 23.8s
Progress: 300/403 frames | FPS: 5.6 | ETA: 18.3s
Progress: 330/403 frames | FPS: 5.6 | ETA: 13.0s
Progress: 360/403 frames | FPS: 5.6 | ETA: 7.7s
Progress: 390/403 frames | FPS: 5.7 | ETA: 2.3s

✓ Processing complete: 403 frames
  Total unique people: 2
  Avg detection: 0.0
  Max detection: 1
  Avg density: 433.7
  Alert frames: 0
  Output saved: /content/output_20251213_074247.mp4

✓ Video saved to: /content/output_20251213_074247.mp4
✓ Also copie

Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://73a823411ceb1940bb.gradio.live


In [17]:
# live webcam with gradio interface...

# ============================================================================
# SECTION 1: INSTALL DEPENDENCIES
# ============================================================================

!pip install -q torch torchvision opencv-python matplotlib ultralytics supervision gradio

# ============================================================================
# SECTION 2: IMPORTS
# ============================================================================

import cv2
import numpy as np
import torch
import torch.nn as nn
from torchvision import models
from matplotlib import cm
from collections import defaultdict
from ultralytics import YOLO
import supervision as sv
import gradio as gr
from PIL import Image
import time

# ============================================================================
# SECTION 3: CONFIGURATION
# ============================================================================

# Check GPU availability
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {DEVICE}")
if DEVICE.type == 'cuda':
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Configuration
ALERT_THRESHOLD = 30
IMAGENET_MEAN = np.array([0.485, 0.456, 0.406], dtype=np.float32)
IMAGENET_STD = np.array([0.229, 0.224, 0.225], dtype=np.float32)

# ============================================================================
# SECTION 4: MODEL ARCHITECTURE
# ============================================================================

def create_csrnet():
    """Create CSRNet model architecture"""
    print("Building CSRNet architecture...")

    vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
    features = list(vgg.features.children())

    frontend = nn.Sequential(*features[0:23])

    backend = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size=3, padding=1, dilation=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(512, 512, kernel_size=3, padding=2, dilation=2),
        nn.ReLU(inplace=True),
        nn.Conv2d(512, 512, kernel_size=3, padding=4, dilation=4),
        nn.ReLU(inplace=True),
        nn.Conv2d(512, 256, kernel_size=3, padding=1, dilation=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(256, 128, kernel_size=3, padding=1, dilation=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(128, 1, kernel_size=1, padding=0),
    )

    model = nn.Sequential(frontend, backend)
    print("✅ CSRNet architecture created")

    return model

def load_trained_model(model_path, device=DEVICE):
    """Load trained CSRNet model"""
    print(f"Loading CSRNet from: {model_path}")

    model = create_csrnet()
    checkpoint = torch.load(model_path, map_location=device)

    if 'model_state' in checkpoint:
        model.load_state_dict(checkpoint['model_state'])
        print(f"✅ Loaded checkpoint from epoch {checkpoint.get('epoch', 'unknown')}")
    else:
        model.load_state_dict(checkpoint)
        print("✅ Loaded model weights")

    model.to(device)
    model.eval()

    print(f"✅ CSRNet ready on {device}")
    return model

# ============================================================================
# SECTION 5: PERSON TRACKER
# ============================================================================

class PersonTracker:
    """Multi-object tracker to prevent re-counting"""

    def __init__(self):
        self.tracker = sv.ByteTrack(
            track_activation_threshold=0.25,
            lost_track_buffer=30,
            minimum_matching_threshold=0.8,
            frame_rate=30
        )

        self.unique_ids = set()
        self.track_history = defaultdict(list)

    def update(self, detections):
        """Update tracker with new detections"""
        tracked_detections = self.tracker.update_with_detections(detections)

        if tracked_detections.tracker_id is not None:
            for track_id in tracked_detections.tracker_id:
                self.unique_ids.add(int(track_id))

        return tracked_detections

    def get_total_unique_count(self):
        """Get total unique people seen"""
        return len(self.unique_ids)

    def reset(self):
        """Reset tracker"""
        self.tracker = sv.ByteTrack(
            track_activation_threshold=0.25,
            lost_track_buffer=30,
            minimum_matching_threshold=0.8,
            frame_rate=30
        )
        self.unique_ids = set()
        self.track_history.clear()

# ============================================================================
# SECTION 6: ENHANCED CROWD COUNTER
# ============================================================================

class EnhancedCrowdCounter:
    """Enhanced counter with detection + tracking + density"""

    def __init__(self, csrnet_model, device=DEVICE, csrnet_input_size=(512, 512)):
        self.csrnet = csrnet_model
        self.device = device
        self.mean = IMAGENET_MEAN
        self.std = IMAGENET_STD

        self.csrnet_input_size = csrnet_input_size
        print(f"✅ CSRNet will process images at: {csrnet_input_size[0]}x{csrnet_input_size[1]}")

        # Load YOLOv8
        print("Loading YOLOv8 person detector...")
        self.yolo = YOLO('yolov8n.pt')
        print("✅ YOLOv8 loaded")

        # Initialize tracker
        self.tracker = PersonTracker()

        # Annotators
        self.box_annotator = sv.BoxAnnotator(thickness=2)
        self.trace_annotator = sv.TraceAnnotator(thickness=2, trace_length=50)

    def preprocess_frame(self, frame):
        """Preprocess frame for CSRNet"""
        frame_resized = cv2.resize(frame,
                                   (self.csrnet_input_size[1], self.csrnet_input_size[0]),
                                   interpolation=cv2.INTER_LINEAR)

        img_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
        img_normalized = img_rgb.astype(np.float32) / 255.0
        img_normalized = (img_normalized - self.mean) / self.std

        img_tensor = torch.from_numpy(img_normalized).permute(2, 0, 1).unsqueeze(0)
        img_tensor = img_tensor.to(self.device, dtype=torch.float32)

        return img_tensor

    def predict_density(self, frame):
        """Get density map from CSRNet"""
        with torch.no_grad():
            img_tensor = self.preprocess_frame(frame)
            density_map = self.csrnet(img_tensor)
            density_np = density_map.squeeze().cpu().numpy()
            count = float(density_np.sum())
        return density_np, count

    def detect_people(self, frame):
        """Detect people using YOLOv8"""
        results = self.yolo(frame, classes=[0], verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        return detections

    def predict_with_tracking(self, frame):
        """Complete prediction pipeline"""
        detections = self.detect_people(frame)
        tracked_detections = self.tracker.update(detections)

        detection_count = len(tracked_detections)
        unique_count = self.tracker.get_total_unique_count()

        density_map, density_count = self.predict_density(frame)

        annotated_frame = frame.copy()

        if len(tracked_detections) > 0:
            annotated_frame = self.box_annotator.annotate(
                scene=annotated_frame,
                detections=tracked_detections
            )

            annotated_frame = self.trace_annotator.annotate(
                scene=annotated_frame,
                detections=tracked_detections
            )

            if tracked_detections.tracker_id is not None:
                for bbox, track_id in zip(tracked_detections.xyxy, tracked_detections.tracker_id):
                    x1, y1, x2, y2 = map(int, bbox)
                    label = f"ID:{track_id}"

                    label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
                    cv2.rectangle(annotated_frame,
                                (x1, y1 - label_size[1] - 10),
                                (x1 + label_size[0] + 10, y1),
                                (0, 255, 0), -1)

                    cv2.putText(annotated_frame, label,
                              (x1 + 5, y1 - 5),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

        return annotated_frame, density_map, detection_count, unique_count, density_count

    def create_heatmap_overlay(self, density_map, original_frame, alpha=0.4):
        """Create density heatmap overlay"""
        h, w = original_frame.shape[:2]
        density_resized = cv2.resize(density_map, (w, h), interpolation=cv2.INTER_CUBIC)

        density_max = density_resized.max()
        if density_max > 0:
            density_normalized = density_resized / density_max
        else:
            density_normalized = density_resized

        heatmap = cm.jet(density_normalized)[:, :, :3]
        heatmap = (heatmap * 255).astype(np.uint8)

        overlay = cv2.addWeighted(original_frame, 1-alpha, heatmap, alpha, 0)

        return overlay

# ============================================================================
# SECTION 7: GRADIO INTERFACE
# ============================================================================

def process_webcam_frame(frame, show_heatmap):
    """Process single webcam frame in real-time"""
    if frame is None:
        return None

    # Convert to BGR for OpenCV processing
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    start_time = time.time()

    # Process frame
    annotated, density_map, det_count, unique_count, dens_count = counter.predict_with_tracking(frame_bgr)

    # Apply heatmap if requested
    if show_heatmap:
        output = counter.create_heatmap_overlay(density_map, annotated, alpha=0.4)
    else:
        output = annotated

    # Calculate FPS
    fps = 1.0 / (time.time() - start_time)

    # Add overlay info panel
    h, w = output.shape[:2]

    # Semi-transparent background for info
    overlay = output.copy()
    cv2.rectangle(overlay, (0, 0), (w, 120), (0, 0, 0), -1)
    cv2.addWeighted(overlay, 0.6, output, 0.4, 0, output)

    # Alert status
    is_alert = det_count > ALERT_THRESHOLD
    if is_alert:
        alert_text = f"ALERT! Count: {det_count} > Threshold: {ALERT_THRESHOLD}"
        color = (0, 0, 255)  # Red
    else:
        alert_text = f"Normal - Count: {det_count}"
        color = (0, 255, 0)  # Green

    # Draw text
    cv2.putText(output, "CSRNet Live Feed", (10, 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(output, alert_text, (10, 55),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    cv2.putText(output, f"Detected: {det_count} | Unique: {unique_count} | Density: {int(dens_count)} | FPS: {fps:.1f}",
                (10, 85), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # Convert back to RGB
    output_rgb = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)

    return output_rgb

def reset_tracker():
    """Reset the tracker"""
    counter.tracker.reset()
    return "✅ Tracker reset successfully!"

# ============================================================================
# SECTION 8: MOUNT GOOGLE DRIVE & LOAD MODEL
# ============================================================================

print("\n" + "="*60)
print("📂 MOUNTING GOOGLE DRIVE")
print("="*60)
print("\nTo use this notebook:")
print("1. Mount your Google Drive (authorize when prompted)")
print("2. Update MODEL_PATH with your model location")
print("3. Use the Gradio interface to test images/videos")
print("\n💡 TIP: Enable GPU in Runtime > Change runtime type > T4 GPU")
print("="*60)

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ Google Drive mounted successfully!")

# ============================================================================
# 🔧 CONFIGURE YOUR MODEL PATH HERE
# ============================================================================
# Update this path to match your model location in Google Drive
# Example paths:
# - '/content/drive/MyDrive/models/best_crowd_counter_unified.pth'
# - '/content/drive/MyDrive/DeepVision/best_crowd_counter_unified.pth'
# - '/content/drive/MyDrive/best_crowd_counter_unified.pth'

MODEL_PATH = '/content/drive/MyDrive/Testing/outputs/best_crowd_counter_unified.pth'  # 👈 UPDATE THIS

print(f"\n📍 Model Path: {MODEL_PATH}")

# Check if model exists
import os
if os.path.exists(MODEL_PATH):
    print(f"✅ Model found! Size: {os.path.getsize(MODEL_PATH) / (1024*1024):.2f} MB")
else:
    print(f"❌ Model not found at: {MODEL_PATH}")
    print("\n💡 TIP: Check your Google Drive and update MODEL_PATH above")
    print("   Right-click on your model file in Drive > Get link to see the path")

# ============================================================================
# SECTION 9: INITIALIZE SYSTEM
# ============================================================================

print("\n" + "="*60)
print("🚀 INITIALIZING SYSTEM")
print("="*60)

# Load model
csrnet = load_trained_model(MODEL_PATH)

# Create counter
counter = EnhancedCrowdCounter(csrnet, csrnet_input_size=(512, 512))

print("\n✅ System ready!")

# ============================================================================
# SECTION 10: LAUNCH GRADIO INTERFACE
# ============================================================================

# Create Gradio interface
with gr.Blocks(title="CSRNet Crowd Counter - Live CCTV") as demo:
    gr.Markdown("""
    # 🎯 CSRNet Crowd Counter - Live Camera Feed
    ### Real-time crowd counting and tracking with density estimation
    **🚀 GPU-Accelerated Live Detection**
    """)

    with gr.Tabs():
        # Live Camera Tab
        with gr.Tab("📹 Live Camera / CCTV Feed"):
            gr.Markdown("""
            ### 🎥 Real-Time Webcam Detection
            - Enable your webcam below
            - Toggle heatmap overlay
            - View live crowd counts with tracking
            - Auto-resets tracker on stream start
            """)

            with gr.Row():
                with gr.Column():
                    webcam_input = gr.Image(sources=["webcam"], streaming=True, label="Live Camera Feed")
                    webcam_heatmap = gr.Checkbox(label="Show Density Heatmap", value=True)

                    with gr.Row():
                        reset_btn = gr.Button("🔄 Reset Tracker", variant="secondary", size="sm")
                        reset_status = gr.Textbox(label="Status", scale=2, show_label=False)

                with gr.Column():
                    webcam_output = gr.Image(label="Processed Live Feed", streaming=True)

                    gr.Markdown("""
                    ### 📊 Live Statistics
                    - **Green** = Normal count (below threshold)
                    - **Red** = Alert! (exceeds threshold)
                    - **Detected**: Current frame detections
                    - **Unique**: Total unique people tracked in session
                    - **Density**: CSRNet density map estimation
                    - **FPS**: Processing speed
                    """)

            # Real-time processing
            webcam_input.stream(
                fn=process_webcam_frame,
                inputs=[webcam_input, webcam_heatmap],
                outputs=webcam_output,
                show_progress="hidden"
            )

            reset_btn.click(fn=reset_tracker, outputs=reset_status)

        # Settings Tab
        with gr.Tab("⚙️ Settings & Info"):
            gr.Markdown("""
            ### Current Configuration
            """)

            with gr.Row():
                with gr.Column():
                    gr.Markdown(f"""
                    **System Info:**
                    - **Device:** {DEVICE}
                    - **GPU:** {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}
                    - **Alert Threshold:** {ALERT_THRESHOLD} people
                    - **CSRNet Input Size:** 512x512
                    - **YOLOv8 Model:** yolov8n.pt
                    """)

                with gr.Column():
                    gr.Markdown("""
                    **Performance Tips:**
                    - Enable GPU for best performance (Runtime > GPU)
                    - Disable heatmap for faster processing
                    - Use good lighting for better detection
                    - Position camera for clear view of people
                    - Reset tracker when starting new monitoring session
                    """)

            gr.Markdown("""
            ---
            ### 🎯 Features:
            - ✅ Real-time person detection & tracking
            - ✅ Density heatmap visualization
            - ✅ Multi-object tracking (prevents re-counting)
            - ✅ Alert system for crowd threshold
            - ✅ FPS monitoring
            - ✅ Session-based unique count tracking

            ### 💡 Use Cases:
            - CCTV monitoring
            - Event crowd management
            - Retail foot traffic analysis
            - Public space monitoring
            - Queue management
            """)

# Launch the interface
print("\n" + "="*60)
print("🚀 LAUNCHING GRADIO INTERFACE")
print("="*60)
print("\n💡 TIPS:")
print("  - Click 'Allow' when browser asks for webcam permission")
print("  - Position camera with good lighting")
print("  - Toggle heatmap on/off as needed")
print("  - Reset tracker when starting new sessions")
print("\n🌐 Interface will open in a new window...")
print("="*60)

demo.launch(share=True)

🚀 Using device: cuda
✅ GPU: Tesla T4
✅ Memory: 15.83 GB

📂 MOUNTING GOOGLE DRIVE

To use this notebook:
1. Mount your Google Drive (authorize when prompted)
2. Update MODEL_PATH with your model location
3. Use the Gradio interface to test images/videos

💡 TIP: Enable GPU in Runtime > Change runtime type > T4 GPU
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

✅ Google Drive mounted successfully!

📍 Model Path: /content/drive/MyDrive/Testing/outputs/best_crowd_counter_unified.pth
✅ Model found! Size: 185.33 MB

🚀 INITIALIZING SYSTEM
Loading CSRNet from: /content/drive/MyDrive/Testing/outputs/best_crowd_counter_unified.pth
Building CSRNet architecture...
✅ CSRNet architecture created
✅ Loaded checkpoint from epoch 191
✅ CSRNet ready on cuda
✅ CSRNet will process images at: 512x512
Loading YOLOv8 person detector...
✅ YOLOv8 loaded

✅ System ready!

🚀 LAUNCHING GRADIO INTERFACE

💡 TIPS:
  - Click 'Allow' when